# Extract

In [ ]:
pip install psycopg2-binary==2.8.6

In [ ]:
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
url =  "https://id.wikipedia.org/wiki/Daftar_miliarder_Forbes"

In [ ]:
def scrape(url):
    logging.info(f"Scraping website with url: '{url}' ...")
    return pd.read_html(url, header=None)

In [ ]:
dfs = scrape(url)[1]

INFO:root:Scraping website with url: 'https://id.wikipedia.org/wiki/Daftar_miliarder_Forbes' ...


In [ ]:
dfs

,No.,Nama,Kekayaan bersih (USD),Usia,Kebangsaan,Sumber kekayaan
0,NaN,Jeff Bezos,$112.0 miliar,54,Amerika Serikat,Amazon.com
1,NaN,Bill Gates,$90.0 miliar,62,Amerika Serikat,Microsoft
2,NaN,Warren Buffett,$84.0 miliar,87,Amerika Serikat,Berkshire Hathaway
3,NaN,Bernard Arnault,$72.0 miliar,69,Prancis,LVMH
4,NaN,Mark Zuckerberg,$71.0 miliar,33,Amerika Serikat,Facebook
5,NaN,Amancio Ortega,$70.0 miliar,81,Spanyol,"Inditex, Zara"
6,NaN,Carlos Slim,$67.1 miliar,78,Meksiko,"América Móvil, Grupo Carso"
7,NaN,Charles Koch,$60.0 miliar,82,Amerika Serikat,Koch Industries
8,NaN,David Koch,$60.0 miliar,77,Amerika Serikat,Koch Industries
9,NaN,Larry Ellison,$58.5 miliar,73,Amerika Serikat,Oracle Corporation


# Transform

In [ ]:
import re

In [ ]:
def transform_money_format(string_money):
    half_clean_string = string_money.lower().replace(",", ".").replace(" ", "").replace("$", "")
    return re.sub(r"[?\[M\]miliar]", "", half_clean_string)

In [ ]:
def transform(df, tahun):
    logging.info("Transforming DataFrame ...")

    columns_mapping = {
        "No.": "nomor_urut",
        "Nama": "nama",
        "Kekayaan bersih (USD)": "kekayaan_bersih_usd",
        "Usia": "usia",
        "Kebangsaan": "kebangsaan",
        "Sumber kekayaan": "sumber_kekayaan"
    }

    renamed_df = df.rename(columns=columns_mapping)
    renamed_df["tahun"] = tahun
    renamed_df["perusahaan"] = renamed_df["sumber_kekayaan"]
    renamed_df["nomor_urut"] = renamed_df.index+1
    renamed_df["kekayaan_bersih_usd_juta"] = renamed_df["kekayaan_bersih_usd"].apply(
        lambda value: float(transform_money_format(value)) * 1000
    )

    return renamed_df[["nama", "nomor_urut", "kekayaan_bersih_usd_juta", "perusahaan", "usia", "kebangsaan", "sumber_kekayaan", "tahun"]]

In [ ]:
df_2018 = transform(dfs, 2018)

INFO:root:Transforming DataFrame ...


In [ ]:
df_2018

,nama,nomor_urut,kekayaan_bersih_usd_juta,perusahaan,usia,kebangsaan,sumber_kekayaan,tahun
0,Jeff Bezos,1,112000.0,Amazon.com,54,Amerika Serikat,Amazon.com,2018
1,Bill Gates,2,90000.0,Microsoft,62,Amerika Serikat,Microsoft,2018
2,Warren Buffett,3,84000.0,Berkshire Hathaway,87,Amerika Serikat,Berkshire Hathaway,2018
3,Bernard Arnault,4,72000.0,LVMH,69,Prancis,LVMH,2018
4,Mark Zuckerberg,5,71000.0,Facebook,33,Amerika Serikat,Facebook,2018
5,Amancio Ortega,6,70000.0,"Inditex, Zara",81,Spanyol,"Inditex, Zara",2018
6,Carlos Slim,7,67100.0,"América Móvil, Grupo Carso",78,Meksiko,"América Móvil, Grupo Carso",2018
7,Charles Koch,8,60000.0,Koch Industries,82,Amerika Serikat,Koch Industries,2018
8,David Koch,9,60000.0,Koch Industries,77,Amerika Serikat,Koch Industries,2018
9,Larry Ellison,10,58500.0,Oracle Corporation,73,Amerika Serikat,Oracle Corporation,2018


# Load

In [ ]:
from sqlalchemy import create_engine

In [ ]:
DB_NAME = "web_scraping_db"
DB_USER = "username"
DB_PASSWORD = "secret"
DB_HOST = "34.71.186.247"
DB_PORT = "5432"
CONNECTION_STRING = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
TABLE_NAME = "almer_orang_terkaya_forbes"

In [ ]:
def write_to_postgres(df, db_name, table_name, connection_string):
    engine = create_engine(connection_string)

    logging.info(f"Writing dataframe to database: '{db_name}', table: '{table_name}' ...")
    df.to_sql(name = table_name, con=engine, if_exists="replace", index=False)

In [ ]:
write_to_postgres(df=df_2018, db_name=DB_NAME, table_name=TABLE_NAME, connection_string=CONNECTION_STRING)

INFO:root:Writing dataframe to database: 'web_scraping_db', table: 'almer_orang_terkaya_forbes' ...
